In [1]:
EMPTY = 0
X = 1
O = 2
N = 3

POSITIVE_DIAGONAL = set(zip(range(N), range(N)))
NEGATIVE_DIAGONAL = set(zip(range(N), reversed(range(N))))

In [2]:
def play_move(board, player, row, col):
    board[row][col] = player
    if check_row(board, row) or check_col(board, col) \
        or check_positive_diagonal(board, row, col) or check_negative_diagonal(board, row, col):
        return 1
    
    return 0

In [3]:
def switch(player):
    if player == X:
        return O
    return X

In [4]:
def open_positions(board):
    positions = []
    for row in range(N):
        for col in range(N):
            if board[row][col] == EMPTY:
                positions.append((row,col))
                
    return positions

In [5]:
def check_row(board, row):
    value = board[row][0]
    for i in range(1, N):
        if board[row][i] != value:
            return False
    return True

In [6]:
def check_col(board, col):
    value = board[0][col]
    for i in range(1, N):
        if board[i][col] != value:
            return False
    return True

In [7]:
def check_positive_diagonal(board, row, col):
    if (row,col) not in POSITIVE_DIAGONAL:
        return False
    
    value = board[row][col]
    for (row, col) in  POSITIVE_DIAGONAL:
        if board[row][col] != value:
            return False
    return True

In [8]:
def check_negative_diagonal(board, row, col):
    if (row,col) not in NEGATIVE_DIAGONAL:
        return False
    
    value = board[row][col]
    for (row, col) in  NEGATIVE_DIAGONAL:
        if board[row][col] != value:
            return False
    return True

In [16]:
import random
from copy import deepcopy

def montecarlo(board, player, N):
    current_player = player
    player_score = 0.0

    for _ in range(N):
        board_copy = deepcopy(board)
        while True:
            if len(open_positions(board_copy)) == 0:
                break

            (row, col) = random.choice(open_positions(board_copy))
            score = play_move(board_copy, player, row, col)
            if score == 1 and player == current_player:
                player_score += 1
                break
            
            player = switch(player)
            
    return player_score / N

In [31]:
def choose(board, player, depth, N=100):
    value = -float('inf')
    
    for row, col in open_positions(board):
        play_move(board, player, row, col)
        next_value = -minimax(board, switch(player), depth-1, N)
        board[row][col] = EMPTY

        if value < next_value:
            value = next_value
            best_move = (row, col)
    
    print(value)
    return best_move

def minimax(board, player, depth, N):
    if len(open_positions(board)) == 0: 
        return 0
    
    if depth == 0:
        return montecarlo(board, player, N)
    
    for row, col in open_positions(board):
        score = play_move(board, player, row, col)
        board[row][col] = EMPTY
        if score == 1:
            return 1
        
    value = -float('inf')
    
    for row, col in open_positions(board):
        play_move(board, player, row, col)
        next_value = -minimax(board, switch(player), depth-1, N)
        board[row][col] = EMPTY
        
        if value < next_value:
            value = next_value

    return value

In [32]:
def print_board(board):
    print()
    for i,row in enumerate(board):
        for square in row:
            if square == X:
                print('  X  |', end='  ')
            elif square == O:
                print('  O  |', end='  ')
            else:
                print('     |', end='  ')
        if i == len(board) - 1:
            print()
        else:
            print('\n'+'________'*N)

In [33]:
def game():
    
    board = [[0 for col in range(N)] for row in range(N)]
    
    player = O
    
    print_board(board)

    while True:
        row, col = choose(board, player, depth=1, N=1)
        score = play_move(board, player, row, col)
        if score == 1:
            print_board(board)
            return player
        
        print_board(board)
        
        if len(open_positions(board)) == 0:
            print('Draw')
            return
        
        player = switch(player)

In [34]:
game()


     |       |       |  
________________________
     |       |       |  
________________________
     |       |       |  
-0.1

     |       |       |  
________________________
     |    O  |       |  
________________________
     |       |       |  
-0.6

     |    X  |       |  
________________________
     |    O  |       |  
________________________
     |       |       |  
-0.0

     |    X  |    O  |  
________________________
     |    O  |       |  
________________________
     |       |       |  
-0.6

     |    X  |    O  |  
________________________
     |    O  |       |  
________________________
  X  |       |       |  
-0.1

     |    X  |    O  |  
________________________
  O  |    O  |       |  
________________________
  X  |       |       |  
-0.3

     |    X  |    O  |  
________________________
  O  |    O  |    X  |  
________________________
  X  |       |       |  
-0.0

  O  |    X  |    O  |  
________________________
  O  |    O  |    X  |  
_______